In [1]:
# 이제 전부 통합

#1 다트에서 파일을 가져오려면 기업명, 기간 등을 입력하고 
# 그 결과로 재무제표 파일을 가져올 수 있는 함수를 만들어야 함.

#2 함수에 필요한 것들 중 파일을 불러올 수 있으려면 rcp, dcm 등이 필요

#3 기업코드도 불러올 수 있어야 함

In [7]:
#1

import requests
import pandas as pd
from io import BytesIO

def dart_download_excel(company, period, rcp, dcm):
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47'
    headers = {'user-agent' : user_agent}
    
    for i in range(min(len(period), len(rcp), len(dcm))):
        url_rcpdcm = 'http://dart.fss.or.kr/pdf/download/excel.do?rcp_no='+rcp[i]+'&dcm_no='+dcm[i]+'&lang=ko'
        sheets = ['연결 재무상태표', '연결 손익계산서', '연결 포괄손익계산서']
        # sheets의 경우 지금은 이렇게 하지만 나중엔 어떤 sheet가 있는지도 알아낼 수 있어야 할 것 같음
        
        for sheet in sheets:
            resp_rcp = requests.get(url_rcpdcm, headers = headers)
            output = BytesIO(resp_rcpdcm.content)
            
            table = pd.read_excel(output, sheet_name = sheet, skiprows = 5)
            table.to_csv(period[i]+company+sheet+".csv", encoding = 'euc-kr')

In [96]:
#2

import requests
import pandas as pd
from io import BytesIO
import re

def find_info(corp_code):

    api_key = '7d523a1a3ce98201eb9eeb5c04c206363c176810'
    url = 'https://opendart.fss.or.kr/api/list.xml?crtfc_key={}&corp_code={}&bgn_de=19990101&last_reprt_at=N&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003&page_count=40'.format(api_key, corp_code)
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47'
    headers = {'user-agent' : user_agent}

    resp = requests.get(url)
    resp_decode = resp.content.decode('utf-8')

    period = re.findall(r'<report_nm>(.*?)</report_nm>', resp_decode)
    rcp = re.findall(r'<rcept_no>(.*?)</rcept_no>', resp_decode)
    # <report_nm> 이런건 미리 해보고 알 수 있는 것들
    # company는 아직... 좀 더 해보고...
    

    dcm = []

    for i in range(len(rcp)):
        url_rcp = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + rcp[i]
        resp_rcp = requests.get(url_rcp, headers = headers)
        resp_rcp_decode = resp_rcp.content.decode('utf-8')
        dcm_i = re.findall(rcp[i]+"', '(.*?)',", resp_rcp_decode)[0]
        dcm = dcm + [dcm_i]
        
    return dict(zip(zip(period,rcp),dcm))
    
def dart_download_excel(company, period, rcp, dcm):
    
    for i in range(min(len(period), len(rcp), len(dcm))):
        url_rcpdcm = 'http://dart.fss.or.kr/pdf/download/excel.do?rcp_no='+rcp[i]+'&dcm_no='+dcm[i]+'&lang=ko'
        sheets = ['연결 재무상태표', '연결 손익계산서', '연결 포괄손익계산서']
        # sheets의 경우 지금은 이렇게 하지만 나중엔 어떤 sheet가 있는지도 알아낼 수 있어야 할 것 같음
        
        resp_rcpdcm = requests.get(url_rcpdcm, headers = headers)
       
        for sheet in sheets:
            output = BytesIO(resp_rcpdcm.content)
            table = pd.read_excel(output, sheet_name = sheet, skiprows = 5)
            table.to_csv(period[i]+company+sheet+".csv", encoding = 'euc-kr')

In [97]:
find_info('00126380')

{('분기보고서 (2020.09)', '20201116001248'): '7549051',
 ('반기보고서 (2020.06)', '20200814001766'): '7446167',
 ('분기보고서 (2020.03)', '20200515001451'): '7341446',
 ('사업보고서 (2019.12)', '20200330003851'): '7202455',
 ('분기보고서 (2019.09)', '20191114001273'): '6958001',
 ('반기보고서 (2019.06)', '20190814002218'): '6846651',
 ('분기보고서 (2019.03)', '20190515001605'): '6738798',
 ('사업보고서 (2018.12)', '20190401004781'): '6616741',
 ('분기보고서 (2018.09)', '20181114001530'): '6382016',
 ('반기보고서 (2018.06)', '20180814001113'): '6282263',
 ('분기보고서 (2018.03)', '20180515001699'): '6180129',
 ('사업보고서 (2017.12)', '20180402005019'): '6060273',
 ('분기보고서 (2017.09)', '20171114001694'): '5845320',
 ('반기보고서 (2017.06)', '20170814001832'): '5752868',
 ('분기보고서 (2017.03)', '20170515003806'): '5653406',
 ('사업보고서 (2016.12)', '20170331004518'): '5540271',
 ('분기보고서 (2016.09)', '20161114001491'): '5349987',
 ('반기보고서 (2016.06)', '20160816001779'): '5261457',
 ('분기보고서 (2016.03)', '20160516003174'): '5146351',
 ('사업보고서 (2015.12)', '201603300

In [79]:
#3

import requests
import pandas as pd
from io import BytesIO
import re
from zipfile import ZipFile
import xmltodict
import json

api_key = '7d523a1a3ce98201eb9eeb5c04c206363c176810'

def find_corp_code(corp_name):

    url_corp = 'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={}'.format(api_key)
    resp_corp = requests.get(url_corp)

    zf_corp = ZipFile(BytesIO(resp_corp.content))
    zf_corp_name = zf_corp.namelist()
    zf_corp_name_pop = zf_corp_name.pop()

    data_xml = zf_corp.read(zf_corp_name_pop).decode('utf-8')
    data_odict = xmltodict.parse(data_xml)
    data_dict = json.loads(json.dumps(data_odict))
    data = data_dict.get('result', {}).get('list')

    where_name = (item for item in data if item['corp_name'] == corp_name)
    # corp_name 이 corp_name인 dict를 list에서 가져오기
    # 만약 값이 없을 경우 false가 dict에 저장됨
    dict_name = next(where_name, False)
    corp_code = dict_name['corp_code']
    
    return corp_code

In [91]:
find_corp_code('카카오')

'00918444'